In [1]:
#### Projeto: Desafio ZAP
#### Programa para Preparar a Base de Treino do ZAP (somente para Apartamentos à Venda)
#### Autor: Rodolfo Bugarin

In [2]:
import pandas as pd
import numpy as np

In [3]:
#
# Carregar o Dataframe
#

In [4]:
import pickle

In [5]:
df = pickle.load(open('source-4-ds-train.pickle', 'rb'))

In [6]:
#
# Tratamento de Dados
#

In [7]:
# A Coluna Target/Explicada passa a ser a 1a coluna
df.insert(loc=0, column='Preco_Venda', value=0)
df['Preco_Venda'] = df.pricinginfos_price
df.drop(['pricinginfos_price'], axis=1, inplace=True)

In [8]:
# O Desafio ZAP pede estimar um preço de venda para os apartamentos no dataset de teste 
# Desta forma, para iremos gerar o modelo com base no dataset de treino somente para os imóveis "apartamento"
# e que estejam à venda (ou ambos)

In [9]:
df.shape

(133964, 37)

In [10]:
df.drop(df[df['pricinginfos_businesstype'] == 'RENTAL'].index, inplace = True) 

In [11]:
df.drop(df[df['unittypes'] != 'APARTMENT'].index, inplace = True) 

In [12]:
df.shape

(64146, 37)

In [13]:
# Criar a coluna Zona com base na coluna address_locationid

# A coluna address_zone está inconsistente. A maioria dos imóveis não tem essa coluna preenchida
# Entretanto, a coluna address_locationid possui essa informação para a maioria dos imóveis.

In [14]:
df.groupby('address_zone').address_zone.count()

address_zone
              56488
Centro         1564
Zona Leste     1185
Zona Norte     1336
Zona Oeste     1039
Zona Sul       2534
Name: address_zone, dtype: int64

In [15]:
import re 
  
# Função para extrair zonas de São Paulo 
def Zona_names(Zona_name): 
    if re.search('\BR>Sao Paulo>NULL>Sao Paulo>.*', Zona_name): 
        Zona_name = Zona_name[28:]
        if re.search('\>.*', Zona_name):
            pos = re.search('\>.*', Zona_name).start() 
            Zona_name = Zona_name[:pos]
        return Zona_name
    else: 
        return "" 
    
# Criação da nova coluna Zona 
df['Zona'] = df['address_locationid'].apply(Zona_names) 
  
# Print the updated dataframe 
df[['address_locationid', 'Zona']].head()

,address_locationid,Zona
3,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Vila Olimpia,Zona Sul
4,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Paraiso,Zona Sul
6,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Pinheiros,Zona Oeste
9,BR>Sao Paulo>NULL>Sao Paulo>Centro>Aclimacao,Centro
10,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Morumbi,Zona Sul


In [16]:
df.groupby('Zona').Zona.count()

Zona
                 44
Centro         6621
Zona Leste    11457
Zona Norte    11401
Zona Oeste     9475
Zona Sul      25148
Name: Zona, dtype: int64

In [17]:
df[df['Zona'] ==""].groupby('address_zone').address_zone.count()

address_zone
              38
Zona Leste     1
Zona Oeste     5
Name: address_zone, dtype: int64

In [18]:
# Nos casos em que a nova coluna "Zona" ficou com contéudo "", preencher com o contéu da coluna "address_zone"
for item in df.iterrows():
    if item[1].Zona == "":
        df.loc[item[0], 'Zona'] = item[1].address_zone

In [19]:
# Nos casos em que a nova coluna "Zona" está preenhida com "Centro", alterar para "Zona Centro"
df['Zona'] = df['Zona'].apply(lambda x: "Zona Centro" if x == 'Centro' else x)

In [20]:
print(df.groupby('Zona').Zona.count())

Zona
                  38
Zona Centro     6621
Zona Leste     11458
Zona Norte     11401
Zona Oeste      9480
Zona Sul       25148
Name: Zona, dtype: int64


In [21]:
#
# Converter as colunas categóricas em Dummies
#

In [22]:
import re

#### Função para remover os espaços e caracteres especiais.
def arrumar_string(v_string):
    novo_string = v_string.replace(' ', '_')
    novo_string = novo_string.replace('.', '')
    novo_string = novo_string.replace('(', '')
    novo_string = novo_string.replace(')', '')
    novo_string = novo_string.replace('-', '')
    novo_string = re.sub("\d", "x", novo_string)
    return novo_string

In [23]:
df.pricinginfos_businesstype = df.pricinginfos_businesstype.apply(arrumar_string)
df['pricinginfos_businesstype'] = df['pricinginfos_businesstype'].apply(lambda x: "SemBusinessDefinido" if x == "" else x)
df_aux = pd.get_dummies(df['pricinginfos_businesstype'])
df = pd.concat([df, df_aux], axis=1)

In [24]:
df.publicationtype = df.publicationtype.apply(arrumar_string)
df['publicationtype'] = df['publicationtype'].apply(lambda x: "SemPublicationDefinido" if x == "" else x)
df_aux = pd.get_dummies(df['publicationtype'])
df = pd.concat([df, df_aux], axis=1)

In [25]:
df.Zona = df.Zona.apply(arrumar_string)
df['Zona'] = df['Zona'].apply(lambda x: "SemZonaDefinida" if x == "" else x)
df_aux = pd.get_dummies(df['Zona'])
df = pd.concat([df, df_aux], axis=1)

In [26]:
#
# Eliminar as colunas que não serão utilizadas no modelo
#

In [27]:
# Todas os imóveis são da cidade de São Paulo, então podemos remover cidade, estado e país
(df.drop(['address_city', 'address_country', 'address_state'], axis=1, inplace=True))

In [28]:
# Eliminar as colunas  tem quase todos os valores vazios
(df.drop(['address_district', 'pricinginfos_period'], axis=1, inplace=True))

In [29]:
# A coluna "address_zone" e "address_locationid" não são mais necessária, pois foram substituídas pela coluna "Zona"
(df.drop(['address_zone', 'address_locationid'], axis=1, inplace=True))

In [30]:
# Utilizaremos a latitude e longitude como vetor de localização do imóvel, então todos os campos de endereço podem ser removidos 
(df.drop(['address_neighborhood', 'address_street', 'address_streetnumber', 'address_unitnumber', 'address_zipcode'], axis=1, inplace=True))

In [31]:
# Eliminar as colunas de contrale e id por serem inúteis ao modelo
(df.drop(['createdat', 'id', 'owner', 'publisherid', 'updatedat', 'address_geolocation_precision'], axis=1, inplace=True))

In [32]:
# Eliminar as colunas de descrição e de links
(df.drop(['description', 'images', 'title'], axis=1, inplace=True))

In [33]:
# Eliminar as colunas que têm valores únicos 
(df.drop(['listingstatus', 'unittypes'], axis=1, inplace=True))

In [34]:
# Eliminar as colunas categóricas
(df.drop(['pricinginfos_businesstype', 'publicationtype', 'Zona'], axis=1, inplace=True))

In [35]:
#
# Separando os dataset source-4-ds-train em "train" e "test"
#

# Nota: a base "test" será utlizada para medir a precisão do modelo. 
#        Essa base "test" não deve ser confunida com o dataset source-4-ds-test

In [36]:
features = df.columns.tolist()[1:]

In [37]:
features

['address_geolocation_location_lat',
 'address_geolocation_location_lon',
 'bathrooms',
 'bedrooms',
 'parkingspaces',
 'pricinginfos_monthlycondofee',
 'pricinginfos_rentaltotalprice',
 'pricinginfos_yearlyiptu',
 'suites',
 'totalareas',
 'usableareas',
 'SALE',
 'PREMIUM',
 'STANDARD',
 'SemZonaDefinida',
 'Zona_Centro',
 'Zona_Leste',
 'Zona_Norte',
 'Zona_Oeste',
 'Zona_Sul']

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X = df[features]
y = df.Preco_Venda

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =42)

In [41]:
# DF para Guardar Medidas de Posicao de São Paulo
df_medidas = pd.DataFrame({'Cidade': ['São Paulo']})

In [42]:
#
# Corrigindo os Missing e Outliers
#

In [43]:
features = df.columns.tolist()[1:12]

In [44]:
tam = len(df_medidas.columns)

In [45]:
# Corrigindo Missings
for i in features:
    mediana = X_train[i].median()
    X_train.loc[X_train[i].isnull(), i] = mediana
    X_test.loc[X_test[i].isnull(), i] = mediana
    X.loc[X[i].isnull(), i] = mediana

    # Guardar Medidas de Posicao
    coluna = i + '_mediana'
    df_medidas.insert(loc = tam, column=coluna, value=mediana)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [46]:
# Corrigindo Outliers
for i in features:
    p99 = np.nanpercentile(X_train[i], q=99.9)
    X_train.loc[X_train[i] > p99, i] = p99
    X_test.loc[X_test[i] > p99, i] = p99
    X.loc[X[i] > p99, i] = p99
    
    # Guardar Medidas de Posicao
    coluna = i + '_p99'
    df_medidas.insert(loc = tam, column=coluna, value=p99)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [47]:
df_medidas.head()

,Cidade,usableareas_p99,totalareas_p99,suites_p99,pricinginfos_yearlyiptu_p99,pricinginfos_rentaltotalprice_p99,pricinginfos_monthlycondofee_p99,parkingspaces_p99,bedrooms_p99,bathrooms_p99,...,totalareas_mediana,suites_mediana,pricinginfos_yearlyiptu_mediana,pricinginfos_rentaltotalprice_mediana,pricinginfos_monthlycondofee_mediana,parkingspaces_mediana,bedrooms_mediana,bathrooms_mediana,address_geolocation_location_lon_mediana,address_geolocation_location_lat_mediana
0,São Paulo,753.0,10736.03,5.0,50801.445,0.0,30622.6,8.0,5.0,8.0,...,86.0,1.0,72.0,0.0,482.0,1.0,3.0,2.0,-46.653233,-23.560326


In [48]:
#
# Criar a coluna que mede a distância entre a geolocalização de referência (mediana das coordenadas dos imóveis à venda na 
# cidade) e cada imóvel listado.
#

In [49]:
# Funçao para calcular a distância entre dois pontos
from math import radians, degrees, sin, cos, asin, acos, sqrt, atan2

def great_circle(lat1, lon1, lat2, lon2):

    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [50]:
lat_mediana = df_medidas.iloc[0]['address_geolocation_location_lat_mediana']

In [51]:
lon_mediana = df_medidas.iloc[0]['address_geolocation_location_lon_mediana']

In [52]:
# Criar a nova coluna que mede a distância do imóvel para a mediana

def calculo_distancia (r):
    v_distancia = great_circle(lat_mediana, lon_mediana, r.address_geolocation_location_lat, r.address_geolocation_location_lon)
    return v_distancia

In [53]:
X_train['Distancia'] = X_train.apply(calculo_distancia, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
X_test['Distancia'] = X_test.apply(calculo_distancia, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
X['Distancia'] = X.apply(calculo_distancia, axis=1)

In [56]:
X_train[['address_geolocation_location_lat', 'address_geolocation_location_lon', 'Distancia']].head()

,address_geolocation_location_lat,address_geolocation_location_lon,Distancia
9669,-23.658375,-46.742390,14.195457
22151,-23.623977,-46.729615,10.523507
15864,-23.617532,-46.759563,12.568512
119694,-23.498732,-46.619448,7.668673
58317,-23.541941,-46.663567,2.300469


In [57]:
# Conferindo a aplicação do cálculo no dataframe:

print (great_circle(lat_mediana, lon_mediana, -23.658375, -46.742390))
print (great_circle(lat_mediana, lon_mediana, -23.623977, -46.729615))
print (great_circle(lat_mediana, lon_mediana, -23.617532, -46.759563))
print (great_circle(lat_mediana, lon_mediana, -23.498732, -46.619448))
print (great_circle(lat_mediana, lon_mediana, -23.541941, -46.663567))

14.195457263644874
10.523507229454507
12.568511926396441
7.668672885666033
2.300468722918037


In [58]:
X_train.drop(['address_geolocation_location_lat', 'address_geolocation_location_lon'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [59]:
X_test.drop(['address_geolocation_location_lat', 'address_geolocation_location_lon'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [60]:
X.drop(['address_geolocation_location_lat', 'address_geolocation_location_lon'], axis=1, inplace=True)

In [61]:
X_test.head()

,bathrooms,bedrooms,parkingspaces,pricinginfos_monthlycondofee,pricinginfos_rentaltotalprice,pricinginfos_yearlyiptu,suites,totalareas,usableareas,SALE,PREMIUM,STANDARD,SemZonaDefinida,Zona_Centro,Zona_Leste,Zona_Norte,Zona_Oeste,Zona_Sul,Distancia
133869,2.0,1.0,2.0,482.0,0.0,72.0,1.0,86.0,78.0,1,0,1,0,0,0,0,1,0,4.047358
60627,2.0,3.0,0.0,665.0,0.0,770.0,1.0,86.0,87.0,1,0,1,0,0,1,0,0,0,8.150184
2179,0.0,2.0,2.0,350.0,0.0,0.0,1.0,65.0,65.0,1,1,0,0,0,0,0,0,1,5.740084
51449,3.0,3.0,4.0,7.0,0.0,0.0,3.0,86.0,417.0,1,0,1,0,0,0,0,1,0,5.765512
42086,6.0,4.0,6.0,0.0,0.0,0.0,4.0,0.0,386.0,1,0,1,0,0,0,0,1,0,4.044988


In [62]:
#
# Salvando as Bases
#

In [63]:
X.to_pickle('df_X_features.pickle')
y.to_pickle('df_y_label.pickle')

In [64]:
X_train.to_pickle('df_X_train_features.pickle')
X_test.to_pickle('df_X_test_features.pickle')

In [65]:
y_train.to_pickle('df_y_train_label.pickle')
y_test.to_pickle('df_y_test_label.pickle')

In [66]:
df_medidas.to_pickle('df_medidas.pickle')

In [67]:
df_medidas.head()

,Cidade,usableareas_p99,totalareas_p99,suites_p99,pricinginfos_yearlyiptu_p99,pricinginfos_rentaltotalprice_p99,pricinginfos_monthlycondofee_p99,parkingspaces_p99,bedrooms_p99,bathrooms_p99,...,totalareas_mediana,suites_mediana,pricinginfos_yearlyiptu_mediana,pricinginfos_rentaltotalprice_mediana,pricinginfos_monthlycondofee_mediana,parkingspaces_mediana,bedrooms_mediana,bathrooms_mediana,address_geolocation_location_lon_mediana,address_geolocation_location_lat_mediana
0,São Paulo,753.0,10736.03,5.0,50801.445,0.0,30622.6,8.0,5.0,8.0,...,86.0,1.0,72.0,0.0,482.0,1.0,3.0,2.0,-46.653233,-23.560326


In [68]:
X_train.head()

,bathrooms,bedrooms,parkingspaces,pricinginfos_monthlycondofee,pricinginfos_rentaltotalprice,pricinginfos_yearlyiptu,suites,totalareas,usableareas,SALE,PREMIUM,STANDARD,SemZonaDefinida,Zona_Centro,Zona_Leste,Zona_Norte,Zona_Oeste,Zona_Sul,Distancia
9669,1.0,2.0,1.0,118.0,0.0,8.0,0.0,52.0,52.0,1,0,1,0,0,0,0,0,1,14.195457
22151,2.0,3.0,2.0,1143.0,0.0,280.0,1.0,254.0,132.0,1,0,1,0,0,0,0,0,1,10.523507
15864,1.0,2.0,1.0,322.0,0.0,560.0,1.0,86.0,44.0,1,0,1,0,0,0,0,0,1,12.568512
119694,6.0,4.0,4.0,482.0,0.0,72.0,2.0,146.0,146.0,1,0,1,0,0,0,1,0,0,7.668673
58317,2.0,4.0,3.0,2467.0,0.0,1705.0,4.0,86.0,231.0,1,0,1,0,1,0,0,0,0,2.300469
